In [1]:
import pandas as pd
import numpy as np
import math

#### 加载真实根因

In [2]:
outlier_timestamp = pd.read_csv('../Anomalytime_data_valid.csv')
outlier_timestamp = outlier_timestamp.sort_values(by='timestamp').reset_index(drop=True)
root_cause_set = {}
timestamps = []
sets = []
for ts in outlier_timestamp['timestamp']:
    rcset = pd.read_csv('../anomaly_injection/timestamp_root_cause/%d.csv' % ts, header=None)
    timestamps.append(ts)
    s = []
    for i in range(len(rcset)):
        s.append(rcset.loc[i].tolist())
    sets.append(s)

root_cause_set['timestamp'] = timestamps
root_cause_set['real_set'] = sets
root_cause_set = pd.DataFrame(root_cause_set)
root_cause_set.head()

,timestamp,real_set
0,1536398100000,"[[c6, l3, e04, p26], [c3, p24, i61, l1]]"
1,1536413700000,"[[e04, l3, c2, p05, i27], [e12, p25, l3, i128,..."
2,1536423000000,"[[c6, l3, e04, p26]]"
3,1536435000000,"[[i119, p18], [c6, l4], [i101, c7], [l3, e11],..."
4,1536441900000,"[[e04, l3, c2, p05, i27]]"


#### 加载计算的根因

In [3]:
def transformRootCauseSet(string):
    root_cause_set = []
    sets = string.split(';')
    for s in sets:
        elements = s.split('&')
        root_cause_set.append(elements)
    return root_cause_set

result = pd.read_csv('../result/submit_valid.csv')
result = result.sort_values(by='timestamp').reset_index(drop=True)
result['set'] = result['set'].apply(transformRootCauseSet)
result = result.merge(root_cause_set, on='timestamp', how='left')
result.to_csv('../result/root_cause_set_valid.csv', index=False)

#### 计算F-score

In [4]:
TP = 0
FN = 0
FP = 0
for ts in result['timestamp'].tolist():
    root_cause = result[result['timestamp']==ts]['real_set']
    root_cause_cal = result[result['timestamp']==ts]['set']
    tmp = 0
    for s1 in root_cause:
        for s2 in root_cause_cal:
            if len(s1) == len(s2) and len(set(s1).intersection(set(s2))) == len(s1):
                tmp += 1
                break
    TP += tmp
    FN += len(root_cause) - tmp
    FP += len(root_cause_cal) - tmp
if TP == 0:
    TP += 1
print(TP, FP, FN)
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
FScore = (2 * Precision * Recall)/(Precision + Recall)
print('F-score = %f' % FScore)

1 25 25
F-score = 0.038462
